# Práctica 5. Neural Language Models

Guillermo Segura Gómez

## Entrenamiento de un modelo de lenguaje neuronal

Un modelo neuronal es una estructura computacional que se inspira en la forma en que las neuronas en el cerebro humano procesan información. En su forma más básica, un modelo neuronal consiste en unidades llamadas "neuronas artificiales" o "nodos" que reciben entradas, las procesan y producen salidas. Estos modelos son capaces de aprender patrones complejos a partir de los datos gracias a su capacidad para ajustar los "pesos" de las conexiones entre las neuronas durante el proceso de entrenamiento.

### Perceptrón: El modelo más sencillo
El perceptrón es uno de los modelos neuronales más simples y sirve como base para arquitecturas más complejas. Fue desarrollado por Frank Rosenblatt en 1957. Un perceptrón toma varias entradas binarias o reales $(x_1, x_2, ..., x_n)$, las multiplica por pesos $(w_1, w_2, ..., w_n)$, y suma estos productos. A esta suma se le puede añadir un término de sesgo $(b)$ o *bias*. Luego, esta suma ponderada se pasa por una función de activación, que en el caso más simple podría ser una función escalón que devuelve 1 si la suma ponderada es mayor que un cierto umbral y -1 (o 0) en caso contrario.

La fórmula de un perceptrón para una entrada y una salida sería algo así:

$$ \text{Salida} = f\left(\sum_{i=1}^{n} w_i \cdot x_i + b\right) $$

Donde $f$ es la función de activación.

El perceptrón puede aprender a separar dos clases linealmente separables ajustando sus pesos durante el entrenamiento, usualmente mediante el algoritmo de descenso del gradiente.

### Modelo Neuronal propuesto por Bengio
Yoshua Bengio es uno de los pioneros en el campo del aprendizaje profundo, y ha contribuido al desarrollo de varios tipos de redes neuronales, incluyendo las Redes Neuronales Profundas (Deep Neural Networks, DNN), las Redes Neuronales Recurrentes (Recurrent Neural Networks, RNN) para procesamiento de secuencias, y más recientemente, las Redes Generativas Adversarias (Generative Adversarial Networks, GAN) y las Transformadores (Transformers) para tareas de NLP y generación de imágenes.

Una arquitectura neuronal profundamente influyente propuesta por Bengio y otros es la **Red Neuronal Profunda (DNN)**, que consiste en múltiples capas de neuronas. Cada capa toma las salidas de la capa anterior como entradas y produce nuevas salidas, que luego se pasan a la siguiente capa. Este proceso continúa hasta que se alcanza la capa de salida. Las capas entre la entrada y la salida se llaman "capas ocultas", y cada una puede aprender diferentes representaciones de los datos de entrada. La presencia de múltiples capas ocultas permite que la red aprenda características cada vez más abstractas y complejas a medida que los datos avanzan a través de la red, lo que ha permitido avances significativos en campos como la visión por computadora y el procesamiento del lenguaje natural.

El entrenamiento de estos modelos se realiza generalmente a través de un algoritmo conocido como **propagación hacia atrás** (backpropagation), combinado con algún tipo de optimización, como el descenso de gradiente estocástico, para ajustar los pesos de la red y minimizar una función de pérdida que mide el error de la red.

Cada uno de estos modelos tiene su propia complejidad y aplicabilidad dependiendo de la tarea específica, y el campo del aprendizaje profundo sigue evolucionando con nuevas arquitecturas y técnicas propuestas regularmente.

Vamos a tratar de replicar en esta práctica el modelo propuesto por Bengio.

In [3]:
# General libraries
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace

import matplotlib.pyplot as plt

Librerías para preprocesamiento, que son las que ya hemos estado utilizando

In [4]:
#Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import pandas as pd
import numpy as np

Utilizamos la Liberia `Pytorch`

PyTorch es una biblioteca de código abierto de aprendizaje automático (Machine Learning) para Python, utilizada tanto en investigación como en producción. Fue desarrollada inicialmente por el laboratorio de investigación de inteligencia artificial de Facebook (FAIR), y desde entonces ha ganado una amplia popularidad en la comunidad de investigación y desarrollo en inteligencia artificial (IA).

PyTorch se destaca por varias razones:

1. **Flexibilidad y dinamismo**: Utiliza lo que se llama "gráficos de cálculo dinámico" (también conocidos como "definidos por ejecución"), lo que significa que el gráfico de cálculo se construye sobre la marcha en cada iteración. Esto proporciona una gran flexibilidad y facilita la experimentación y el prototipado, especialmente para modelos complejos y dinámicos.

2. **Facilidad de uso**: Su interfaz es intuitiva, especialmente para quienes están familiarizados con Python y las bibliotecas de cálculo numérico como NumPy. PyTorch también proporciona una gran cantidad de utilidades y abstracciones de alto nivel que simplifican tareas comunes de aprendizaje automático.

3. **Soporte para redes neuronales profundas (Deep Learning)**: PyTorch incluye un módulo `torch.nn` dedicado a las redes neuronales, que ofrece una amplia variedad de capas y funciones de activación predefinidas, facilitando la construcción y entrenamiento de modelos complejos.

4. **Autodiferenciación**: A través de su módulo `torch.autograd`, PyTorch proporciona autodiferenciación, lo que significa que puede calcular automáticamente gradientes o derivadas de operaciones con respecto a las entradas, lo cual es esencial para el algoritmo de retropropagación utilizado en el entrenamiento de redes neuronales.

5. **Soporte para GPU**: PyTorch ofrece un soporte excelente para la computación en GPU (Unidad de Procesamiento Gráfico), lo que permite acelerar significativamente las operaciones matemáticas complejas y el entrenamiento de modelos, especialmente útil para grandes conjuntos de datos y modelos profundos.

6. **Comunidad y ecosistema**: PyTorch goza de una comunidad activa y en crecimiento, que contribuye constantemente con una amplia gama de herramientas y bibliotecas complementarias, tutoriales, y soporte. Esto hace que sea más fácil encontrar recursos y solucionar problemas.

PyTorch se ha convertido en una de las herramientas más importantes para los investigadores y profesionales en el campo del aprendizaje automático y el deep learning, debido a su flexibilidad, facilidad de uso, y robustas capacidades para el desarrollo y entrenamiento de modelos de IA.

In [2]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
#scikit-learn
from sklearn.metrics import accuracy_score

El propósito general de las siguientes líneas es asegurar la reproducibilidad de los experimentos que vamos hacer en ciencia de datos y aprendizaje automático. Al fijar las semillas de los generadores de números aleatorios en las distintas bibliotecas utilizadas, se asegura que el código producirá los mismos resultados cada vez que se ejecute, lo cual es crucial para la depuración, la comparación de modelos y la publicación de resultados científicos reproducibles.

In [6]:
seed = 1111
random.seed(seed) #python seed
np.random.seed(seed) #numpy seed
torch.manual_seed(seed) #torch seed
torch.backends.cudnn.benchmark = False 

1. `seed = 1111`: Esta línea establece un número (en este caso, 1111) como valor de la variable `seed`. Esta variable se utilizará para inicializar los generadores de números aleatorios en diferentes librerías, con el fin de hacer que los resultados sean reproducibles.

2. `random.seed(seed)`: Esta línea inicializa el generador de números aleatorios de Python con la semilla definida en la variable `seed`. Esto significa que las funciones del módulo `random` de Python que generan números aleatorios producirán la misma secuencia de números cuando se ejecuten con la misma semilla.

3. `np.random.seed(seed)`: De manera similar a la anterior, esta línea inicializa el generador de números aleatorios de la biblioteca NumPy con la misma semilla. NumPy es ampliamente utilizado para cálculos numéricos, y su generador de números aleatorios se utiliza, por ejemplo, para inicializar pesos en redes neuronales o para dividir conjuntos de datos de manera aleatoria durante el entrenamiento.

4. `torch.manual_seed(seed)`: Esta línea inicializa el generador de números aleatorios de PyTorch con la misma semilla. PyTorch es una biblioteca para el aprendizaje automático y el deep learning, y utilizar una semilla fija asegura la reproducibilidad de los experimentos, especialmente importante para la inicialización de los pesos de las redes neuronales, entre otros.

5. `torch.backends.cudnn.benchmark = False`: Esta línea desactiva el modo "benchmark" de la biblioteca cuDNN utilizada por PyTorch para operaciones de deep learning en GPUs NVIDIA. El modo "benchmark" busca la configuración óptima para las operaciones de convolución en tu hardware específico, lo cual puede mejorar el rendimiento. Sin embargo, esto puede llevar a resultados no deterministas debido a la selección variable de algoritmos de convolución. Al establecer `benchmark` como `False`, se prioriza la reproducibilidad sobre el rendimiento potencialmente óptimo.

Inicializamos los datos en un data frame de pandas.

In [8]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train.txt"
path_text_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val.txt"

In [9]:
X_train = pd.read_csv(path_text, sep = '\r\n', engine = 'python', header = None).loc[:, 0].values.tolist()
X_val = pd.read_csv(path_text_val, sep = '\r\n', engine = 'python', header = None).loc[:, 0].values.tolist()

Creamos una clase que reciba el tamaño del modleo, el número de tokens máximo y un tokenizador.

La clase `NgramData` definida en el siguiente código está diseñada para procesar datos textuales para tareas de modelado de lenguaje o similares, utilizando un enfoque basado en n-gramas. Un n-grama es una secuencia contigua de \(N$ ítems (en este caso, palabras) de un texto dado. El propósito de esta clase incluye varias funciones importantes para el preprocesamiento de datos textuales y su preparación para algoritmos de aprendizaje automático, especialmente en el contexto del procesamiento del lenguaje natural (NLP). A continuación, desgloso las funcionalidades principales de la clase:

1. **Inicialización (`__init__`)**: Al crear una instancia de `NgramData`, se configuran varios parámetros importantes, como el tamaño de los n-gramas (`N`), el tamaño máximo del vocabulario (`vocab_max`), el tokenizador personalizado (`tokenizer`), y un modelo de embeddings (`embeddings_model`). También se definen algunos tokens especiales como desconocido (`UNK`), inicio de secuencia (`SOS`), y fin de secuencia (`EOS`), y un conjunto de signos de puntuación a ignorar.

2. **Tokenización (`default_tokenizer`)**: La función `default_tokenizer` proporciona una forma simple de dividir un documento en tokens (palabras) usando espacios en blanco. Puede ser reemplazado por un tokenizador personalizado si se proporciona uno durante la inicialización.

3. **Limpieza y preparación del vocabulario (`remove_word`, `get_vocab`, `sortFreqDict`)**: Estas funciones ayudan a limpiar y preparar el vocabulario. `remove_word` decide si una palabra debe ser eliminada basándose en si es un signo de puntuación o un dígito. `get_vocab` construye el vocabulario a partir de un corpus dado, excluyendo las palabras no deseadas y limitando el tamaño del vocabulario. `sortFreqDict` ordena las palabras en el vocabulario por frecuencia.

4. **Ajuste del modelo (`fit`)**: Esta función ajusta el modelo al corpus dado, construyendo el vocabulario, y luego crea mapeos de palabras a identificadores únicos (`w2id`) y viceversa (`id2w`). Si se proporciona un modelo de embeddings, también se construye una matriz de embeddings para las palabras en el vocabulario.

5. **Transformación de los datos (`transform`)**: `transform` convierte un corpus de documentos en un formato adecuado para el modelado, específicamente en pares de entradas y salidas donde cada entrada es una secuencia de \(N-1$ palabras (en forma de identificadores) y la salida es la siguiente palabra en la secuencia.

6. **Generación de n-gramas (`get_ngram_doc`)**: Esta función convierte un documento en una lista de n-gramas, asegurándose de que las palabras desconocidas se reemplacen por el token `UNK` y añadiendo tokens especiales al principio y al final del documento para indicar el inicio y el fin de la secuencia.

7. **Reemplazo de palabras desconocidas (`replace_unk`)**: Reemplaza las palabras que no están en el vocabulario con el token de palabra desconocida (`UNK`).

El propósito general de `NgramData` es facilitar el preprocesamiento de textos para tareas de NLP, especialmente para modelar la secuencia de palabras en un texto utilizando el enfoque de n-gramas, lo que es útil en aplicaciones como la predicción de la siguiente palabra, análisis de sentimientos, clasificación de texto, y más.

In [12]:
args = Namespace()
args.N = 4

class NgramData():
    # Inicialización de la clase NgramData
    def __init__(self, N:int, vocab_max: int = 5000, tokenizer = None, embeddings_model = None):
        # Asignación de un tokenizador personalizado o el predeterminado si no se proporciona
        self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
        # Definición de signos de puntuación a ignorar
        self.punct = set(['.', ',', ';', ':', '-', '_', '!', '¡', '?', '¿', '^', '<url>', '*', '@usuario'])
        # Número de palabras en cada n-grama
        self.N = N
        # Tamaño máximo del vocabulario
        self.vocab_max = vocab_max
        # Token para palabras desconocidas
        self.UNK = '<unk>'
        # Token para indicar el inicio de una secuencia
        self.SOS = '<s>'
        # Token para indicar el final de una secuencia
        self.EOS = '</s>'
        # Modelo de embeddings (opcional)
        self.embeddings_model = embeddings_model

    # Función para obtener el tamaño del vocabulario
    def get_vocab_size(self) -> int:
        return len(self.vocab)

    # Tokenizador predeterminado que divide el texto por espacios
    def default_tokenizer(self, doc: str) -> list:
        return doc.split(" ")
    
    # Función para determinar si una palabra debe eliminarse (basada en puntuación o si es un número)
    def remove_word(self, word: str) -> bool:
        word = word.lower()
        is_punct = word in self.punct
        is_digit = word.isnumeric()
        return is_punct or is_digit
    
    # Construye el vocabulario a partir de un corpus, excluyendo palabras según `remove_word` y limitando el tamaño
    def get_vocab(self, corpus: list) -> set:
        # Construcción de la distribución de frecuencia de palabras
        freq_dist = FreqDist([w.lower() for sentence in corpus for w in self.tokenizer(sentence) if not self.remove_word(w)])
        # Ordenar palabras por frecuencia y limitar el tamaño del vocabulario
        sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3]
        return set(sorted_words)
    
    # Ordena el diccionario de frecuencia de palabras
    def sortFreqDict(self, freq_dist) -> list:
        freq_dict = dict(freq_dist)
        return sorted(freq_dict, key=freq_dict.get, reverse=True)
    
    # Ajusta el modelo al corpus, construyendo vocabulario, mapeos de palabras a ID y opcionalmente una matriz de embeddings
    def fit(self, corpus: list) -> None:
        self.vocab = self.get_vocab(corpus)
        # Agregar tokens especiales al vocabulario
        self.vocab.update({self.UNK, self.SOS, self.EOS})
        
        self.w2id = {}
        self.id2w = {}
        
        # Opcional: inicialización de la matriz de embeddings
        if self.embeddings_model is not None:
            self.embedding_matrix = np.empty([len(self.vocab), self.embeddings_model.vector_size])
            
        id = 0
        for doc in corpus:
            for word in self.tokenizer(doc):
                word_ = word.lower()
                if word_ in self.vocab and word_ not in self.w2id:
                    self.w2id[word_] = id
                    self.id2w[id] = word_
                    # Si se proporciona un modelo de embeddings, asignar vector o vector aleatorio si la palabra no está en el modelo
                    if self.embeddings_model is not None:
                        self.embedding_matrix[id] = self.embeddings_model[word_] if word_ in self.embeddings_model else np.random.rand(self.embeddings_model.vector_size)
                    id += 1
        
        # Actualizar mapeos con tokens especiales
        self.w2id.update({self.UNK: id, self.SOS: id+1, self.EOS: id+2})
        self.id2w.update({id: self.UNK, id+1: self.SOS, id+2: self.EOS})
    
    # Transforma el corpus en secuencias de entrada (X_ngrams) y etiquetas objetivo (y) para el modelado
    def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:
        X_ngrams = []  # Lista para almacenar secuencias de entrada
        y = []  # Lista para almacenar las etiquetas objetivo

        # Iterar sobre cada documento en el corpus
        for doc in corpus:
            # Obtener n-gramas para el documento actual
            doc_ngram = self.get_ngram_doc(doc)
            # Iterar sobre cada ventana de palabras (n-grama) en el documento
            for words_window in doc_ngram:
                # Convertir palabras en IDs usando el mapeo palabra-ID
                words_window_ids = [self.w2id[w] for w in words_window]
                # Las primeras N-1 palabras son la entrada, la última palabra es la etiqueta objetivo
                X_ngrams.append(list(words_window_ids[:-1]))
                y.append(words_window_ids[-1])
        # Convertir las listas en arrays de NumPy para su uso en modelos de aprendizaje automático
        return np.array(X_ngrams), np.array(y)
    
    # Genera n-gramas a partir de un documento, preparándolo para la transformación
    def get_ngram_doc(self, doc: str) -> list:
        # Tokenizar el documento
        doc_tokens = self.tokenizer(doc)
        # Reemplazar tokens desconocidos por el token <unk>
        doc_tokens = self.replace_unk(doc_tokens)
        # Convertir todos los tokens a minúsculas
        doc_tokens = [w.lower() for w in doc_tokens]
        # Añadir tokens de inicio (<s>) y fin (</s>) al documento
        doc_tokens = [self.SOS]*(self.N-1) + doc_tokens + [self.EOS]
        # Generar y retornar n-gramas del documento procesado
        return list(ngrams(doc_tokens, self.N))
    
    # Reemplaza tokens desconocidos en una lista de tokens por el token <unk>
    def replace_unk(self, doc_tokens: list) -> list:
        # Iterar sobre cada token en la lista de tokens
        for i, token in enumerate(doc_tokens):
            # Si el token no está en el vocabulario, reemplazarlo por <unk>
            if token.lower() not in self.vocab:
                doc_tokens[i] = self.UNK
        # Retornar la lista de tokens con los desconocidos reemplazados
        return doc_tokens


In [13]:
tk = TweetTokenizer() # Inicializamos el tokenizador de tweets

# Creamos un objeto con la clase que definimos
ngram_data = NgramData(args.N, 5000, tk.tokenize)
ngram_data.fit(X_train) # Construye el vocabulario

Probamos que funcione el objeto que acabamos de crear. 

In [14]:
print(f'Vocab size: {ngram_data.get_vocab_size()}')

Vocab size: 5000


Ahora utilizamos los métodos de la clase que creamos. El método `transform` transforma los datos textuales proporcionados en un formato que es adecuado para el entrenamiento o la evaluación en modelos de aprendizaje automático.
- El método `transform` procesa este conjunto de datos y lo convierte en dos nuevas estructuras: `X_ngram_train` y `y_ngram_train`.

- `X_ngram_train` contiene las secuencias de entrada transformadas, donde cada entrada es una secuencia de $N-1$ palabras (o tokens) representadas por sus identificadores numéricos. Estas secuencias se utilizan como entradas para tu modelo.
- `y_ngram_train` contiene las etiquetas objetivo asociadas a cada secuencia de entrada en `X_ngram_train`. En el contexto de n-gramas, cada etiqueta es la palabra (o token) que sigue a la secuencia de $N-1$ palabras en el texto original, también representada por su identificador numérico. Estas etiquetas se utilizan como objetivos de predicción para tu modelo.

Luego lo repetimos para el conjunto de validación

In [15]:
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [17]:
X_ngram_train

array([[4998, 4998, 4998],
       [4998, 4998, 4997],
       [4998, 4997, 4997],
       ...,
       [4997,  947,   32],
       [ 947,   32, 2520],
       [  32, 2520, 4997]])

Ya tenemos las listas de entrenamiento para nuestro modelo. Ahora si queremos ver las palabras en lugar de sus tokens, construimos el siguiente vocabulario.

In [18]:
[ngram_data.id2w[w] for w in y_ngram_train[:22]] 

['<unk>',
 '<unk>',
 '<unk>',
 'q',
 'se',
 'puede',
 'esperar',
 'del',
 'maricon',
 'de',
 'closet',
 'de',
 'la',
 'yañez',
 'aun',
 'recuerdo',
 'esa',
 'ves',
 'q',
 'lo',
 'vi',
 'en']

Ahora necesitamos pasar estos datos en tensores de PyTorch y, organizarlos en un DataLoader de PyTorch para facilitar el entrenamiento y la evaluación de modelos de aprendizaje profundo. PyTorch utiliza tensores, que son una generalización de matrices y vectores, como su estructura de datos principal para realizar operaciones de aprendizaje automático, especialmente en redes neuronales.

Para el entrenamiento, es útil organizar los datos en lotes (batches), y PyTorch ofrece una herramienta llamada **DataLoader** para esto. Un DataLoader puede cargar los datos en lotes de un tamaño especificado y puede mezclar los datos para reducir el riesgo de sobreajuste. Para usar un DataLoader, primero debes organizar tus tensores en un Dataset, que es otra abstracción de PyTorch que facilita trabajar con conjuntos de datos.

In [19]:
# Set batch size in args
args.batch_size = 64
# Num workers
args.num_workers = 2

# Convertimos los datos a tensores de pytorch
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype = torch.int64),
                              torch.tensor(y_ngram_train, dtype = torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = True)

val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype = torch.int64),
                            torch.tensor(y_ngram_val, dtype = torch.int64))

val_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = False)

### ¿Qué es un Batch?

Un "batch" o lote es simplemente un subconjunto de tu conjunto de datos de entrenamiento que se utiliza para entrenar el modelo en una sola iteración del algoritmo de optimización, como el descenso de gradiente. En lugar de pasar todo el conjunto de datos a través de la red de una vez (lo que se conoce como entrenamiento por lotes o "batch training") o pasar una sola muestra a la vez (entrenamiento estocástico), pasas un número fijo de muestras. Este número es el tamaño del lote, también conocido como "batch size".

### ¿Por qué usar Batches?

1. **Eficiencia computacional**: Utilizar lotes permite que el proceso de entrenamiento sea más eficiente desde el punto de vista computacional. Las operaciones matriciales en lotes pueden aprovechar mejor las optimizaciones de hardware, como las GPUs, en comparación con el procesamiento de una sola muestra a la vez.

2. **Estabilidad y calidad del entrenamiento**: El entrenamiento con lotes puede ayudar a estabilizar el aprendizaje. Al promediar el gradiente sobre varias muestras, se reduce la varianza en la estimación del gradiente, lo que puede llevar a una convergencia más suave durante el entrenamiento.

3. **Uso de la memoria**: Cargar el conjunto de datos completo en la memoria a la vez puede no ser factible, especialmente para conjuntos de datos grandes. El uso de lotes permite que el modelo se entrene con subconjuntos del conjunto de datos, lo que es más manejable desde el punto de vista del uso de la memoria.

### Trade-offs del Tamaño del Batch

El tamaño del lote es un hiperparámetro que puedes ajustar, y su elección puede afectar la calidad del modelo y el tiempo de entrenamiento:

- **Batch sizes grandes**: Pueden llevar a una estimación más precisa del gradiente, pero pueden requerir más memoria y pueden llevar a un entrenamiento más lento por iteración. Además, batch sizes grandes a veces pueden llevar a un entrenamiento que converge a mínimos menos óptimos debido a la suavización del paisaje del error.

- **Batch sizes pequeños**: Pueden aumentar la varianza en la estimación del gradiente, lo que puede ayudar al modelo a escapar de mínimos locales, potencialmente encontrando mejores mínimos globales. Sin embargo, esto también puede hacer que la trayectoria de entrenamiento sea más ruidosa y posiblemente más lenta en converger. Desde el punto de vista computacional, los lotes más pequeños son menos eficientes.

### Iteración, Época y Batch

Para aclarar más:

- **Iteración**: Cada paso del algoritmo de entrenamiento, en el que el modelo se actualiza, es una iteración. En cada iteración, se utiliza un lote de datos.

- **Época**: Una época completa ocurre cuando cada muestra en el conjunto de datos de entrenamiento ha sido utilizada una vez para actualizar el modelo. Por lo tanto, el número de iteraciones necesarias para completar una época depende del tamaño del conjunto de datos y del tamaño del lote.

In [20]:
batch = next(iter(train_loader))
print(f'X shape: {batch[0].shape}')
print(f'y shape: {batch[1].shape}')

X shape: torch.Size([64, 3])
y shape: torch.Size([64])


Parámetros que definen aspectos estructurales y de regularización de un modelo de red neuronal, que influyen en cómo el modelo aprenderá de los datos y generalizará a nuevos ejemplos no vistos. Utilizamos los parámetros del modelo propuesto por Bengio.

In [21]:
# Vocab size
args.vocab_size = ngram_data.get_vocab_size()

# Dimension of word embeddings
args.d = 50

# Dimension for hidden layer
args.d_h = 100

# Dropout
args.dropout = 0.1

Definimos la clase `NeuralLM`, que representa un Modelo de Lenguaje Neuronal, utilizando PyTorch. Esta clase hereda de `nn.Module`, que es la clase base para todos los módulos de red neuronal en PyTorch, proporcionando funcionalidades útiles como el seguimiento de parámetros, la GPU/CPU transferencia, etc. Veamos los componentes y la funcionalidad de la clase `NeuralLM`:

#### Inicialización (`__init__`)
En el método `__init__`, se inicializan los componentes del modelo:

1. **`self.window_size`**: Define el tamaño de la ventana de entrada para el modelo, basado en el parámetro `N` menos 1. Esto se debe a que, en un modelo de lenguaje basado en n-gramas, si `N` es el tamaño del n-grama, entonces `N-1` palabras se utilizan para predecir la `N`-ésima palabra.

2. **`self.embedding_dim`**: Es la dimensión de los embeddings de palabras, definida por el parámetro `args.d`. Los embeddings de palabras transforman índices de palabras en vectores densos que capturan información semántica.

3. **`self.emb`**: Una capa de embedding que convierte índices de palabras en embeddings. Utiliza el tamaño del vocabulario (`args.vocab_size`) y la dimensión de los embeddings (`args.d`) definidos anteriormente.

4. **`self.fc1`**: La primera capa lineal (o completamente conectada) que transforma la entrada aplanada de la capa de embedding en una representación de dimensión intermedia (`args.d_h`). La entrada de esta capa es el tamaño de la ventana de entrada multiplicado por la dimensión del embedding.

5. **`self.drop1`**: Una capa de Dropout que "apaga" aleatoriamente un porcentaje (`args.dropout`) de las activaciones en la capa anterior para prevenir el sobreajuste.

6. **`self.fc2`**: La segunda capa lineal que transforma la salida de la capa oculta en un vector del tamaño del vocabulario. Esta capa produce las puntuaciones (logits) para cada palabra en el vocabulario, que luego pueden ser convertidas en probabilidades mediante una función softmax. No hay sesgo en esta capa (`bias=False`).

#### Forward Pass (`forward`)
En el método `forward`, se define cómo fluyen los datos a través del modelo:

1. **Embedding**: Primero, los índices de palabras en `x` son transformados en embeddings de palabras mediante `self.emb(x)`.

2. **Flatten**: Los embeddings son aplanados en un vector único para cada muestra en el lote (`x.view(-1, self.window_size*self.embedding_dim)`). Esto es necesario porque las capas lineales esperan entradas de forma plana.

3. **Capa oculta y activación**: La salida aplanada es pasada a través de la primera capa lineal (`self.fc1(x)`) y luego a través de una función de activación ReLU (`F.relu()`), produciendo la representación de la capa oculta `h`.

4. **Dropout**: La representación de la capa oculta pasa a través de la capa de Dropout (`self.drop1(h)`), lo que ayuda a prevenir el sobreajuste reduciendo la dependencia en cualquier neurona individual.

5. **Salida**: Finalmente, la representación de la capa oculta después del Dropout se pasa a través de la segunda capa lineal (`self.fc2(h)`), generando las puntuaciones (logits) para cada palabra en el vocabulario.

Este modelo puede ser entrenado para predecir la siguiente palabra en una secuencia, dado un contexto de `N-1` palabras, lo cual es una tarea común en el modelado del lenguaje. La función de pérdida (como la entropía cruzada) y un optimizador (como el descenso de gradiente estocástico o Adam) se usarían en conjunto con este modelo para entrenarlo en un conjunto de datos específico.

In [23]:
class NeuralLM(nn.Module):
    def __init__(self, args):
        # Inicializa la clase padre, nn.Module
        super(NeuralLM, self).__init__()
        
        # Tamaño de la ventana de entrada (contexto) para el modelo, basado en N-1
        # donde N es el tamaño de los n-gramas considerados.
        self.window_size = args.N-1
        
        # Dimensión de los embeddings de palabras, que transforman índices de palabras
        # en vectores densos que capturan información semántica.
        self.embedding_dim = args.d
        
        # Capa de embedding que convierte índices de palabras en vectores densos.
        # Utiliza el tamaño del vocabulario y la dimensión de los embeddings especificados.
        self.emb = nn.Embedding(args.vocab_size, self.embedding_dim)
        
        # Primera capa lineal que transforma la entrada aplanada de la capa de embedding
        # a una representación de dimensión intermedia especificada por args.d_h.
        self.fc1 = nn.Linear(self.embedding_dim * self.window_size, args.d_h)
        
        # Capa de Dropout que "apaga" aleatoriamente un porcentaje de las activaciones
        # en la capa anterior para prevenir el sobreajuste.
        self.drop1 = nn.Dropout(p=args.dropout)
        
        # Segunda capa lineal que transforma la salida de la capa oculta en un vector
        # del tamaño del vocabulario. Esta capa produce los logits para cada palabra en el vocabulario.
        self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)
        
    def forward(self, x):
        # Transforma los índices de palabras en x en embeddings de palabras.
        x = self.emb(x)
        
        # Aplana los embeddings en un vector único para cada muestra en el lote,
        # preparándolos para la entrada a la capa lineal.
        x = x.view(-1, self.window_size * self.embedding_dim)
        
        # Pasa la entrada aplanada a través de la primera capa lineal y luego
        # a través de una función de activación ReLU.
        h = F.relu(self.fc1(x))
        
        # Aplica Dropout a la representación de la capa oculta para mejorar la generalización.
        h = self.drop1(h)
        
        # La salida final se obtiene pasando la representación de la capa oculta después de Dropout
        # a través de la segunda capa lineal, generando los logits para cada palabra en el vocabulario.
        return self.fc2(h)


Ahora necesitamos mas funciones para poder hacer la evaluación del modelo.

In [24]:
# Esta función toma los logits (es decir, las salidas no normalizadas de un modelo) 
# y devuelve las predicciones de clase como índices.
def get_preds(raw_logits):
    # Calcula las probabilidades aplicando la función softmax a los logits.
    # La operación detach() se usa para evitar que se calculen gradientes para estas operaciones,
    # ya que solo se necesitan las probabilidades para hacer predicciones.
    probs = F.softmax(raw_logits.detach(), dim=1)
    
    # Encuentra el índice de la mayor probabilidad en cada fila (es decir, para cada ejemplo en el lote),
    # que corresponde a la clase predicha. Luego, convierte el tensor a un array de NumPy.
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()
    
    return y_pred


# Evalúa el modelo en un conjunto de datos proporcionado y devuelve la precisión del modelo.
def model_eval(data, model, gpu=False):
    # Desactiva el cálculo de gradientes para acelerar las cosas y reducir el uso de memoria
    # ya que no se necesita para la evaluación.
    with torch.no_grad():
        preds, tgts = [], []  # Listas para almacenar predicciones y etiquetas verdaderas
        
        # Itera sobre los lotes de datos en el DataLoader
        for window_words, labels in data:
            # Si se utiliza GPU, mueve los datos al dispositivo adecuado
            if gpu:
                window_words = window_words.cuda()
                
            # Obtiene los logits del modelo para el lote actual
            outputs = model(window_words)
            
            # Obtiene las predicciones de clase para el lote actual utilizando la función get_preds
            y_pred = get_preds(outputs)
            
            # Extrae las etiquetas verdaderas del lote actual y las convierte a un array de NumPy
            tgt = labels.numpy()
            
            # Almacena las predicciones y las etiquetas verdaderas
            tgts.append(tgt)
            preds.append(y_pred)
        
        # Aplana las listas de listas para obtener una única lista de etiquetas y predicciones
        tgts = [e for l in tgts for e in l]
        preds = [e for l in preds for e in l]
        
        # Calcula y devuelve la precisión del modelo comparando las predicciones con las etiquetas verdaderas
        return accuracy_score(tgts, preds)
    

# Guarda el estado actual del modelo y, si es el mejor modelo hasta el momento según 
# algún criterio, guarda una copia separada.
def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pt"):
    # Construye la ruta completa del archivo donde se guardará el estado del modelo
    filename = os.path.join(checkpoint_path, filename)
    
    # Guarda el estado del modelo en la ruta especificada
    torch.save(state, filename)
    
    # Si el modelo actual es el "mejor" según algún criterio, guarda una copia separada
    if is_best:
        # Copia el archivo del checkpoint al archivo del "mejor modelo"
        shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))



Necesitamos ahora definir todos los hiperparámetros de la red.

In [25]:
# Model hyperparameters

args. vocab_size = ngram_data.get_vocab_size()
args.d = 100
args.d_h = 200
args.dropout = 0.1

# Training hyperparameters
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

# Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

# Saving directoty
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

# Create model
model = NeuralLM(args)

# Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()
    
# Loss, optimizer an scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 'min',
                patience=args.lr_patience,
                verbose=True,
                factor=args.lr_factor
                )

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Ahora realizamos el entrenamiento y validación del modelo de aprendizaje profundo utilizando PyTorch, incluyendo características como el early stopping y el ajuste dinámico de hiperparámetros (scheduler). Aquí está lo que sucede durante el proceso:

1. **Inicialización**: Se establece un tiempo de inicio para medir la duración del entrenamiento, se inicializa la mejor métrica a 0 para seguimiento, y se crean listas para almacenar la historia de las métricas durante el entrenamiento y la validación.

2. **Bucle de Entrenamiento**: Para cada época dentro del número total de épocas especificadas:
   
   a. Se inicializa el modelo en modo de entrenamiento.
   
   b. Se itera sobre los lotes de datos del conjunto de entrenamiento (`train_loader`). Para cada lote, se mueven los datos al dispositivo adecuado (GPU si está disponible), se realiza un forward pass para obtener las predicciones del modelo, y se calcula la pérdida utilizando una función de pérdida (`criterion`) especificada.
   
   c. Se calcula una métrica de entrenamiento (precisión) comparando las predicciones con las etiquetas verdaderas, y se almacena para análisis posterior.
   
   d. Se ejecuta el backward pass para calcular los gradientes de la pérdida respecto a los parámetros del modelo, seguido de un paso de optimización para actualizar los parámetros del modelo utilizando el optimizador especificado. Se limpian los gradientes antes de cada backward pass para evitar la acumulación.

3. **Evaluación y Ajuste**: Después de cada época de entrenamiento:
   
   a. Se evalúa el modelo en el conjunto de validación utilizando la función `model_eval`, que desactiva el cálculo de gradientes y devuelve la precisión en el conjunto de validación.
   
   b. Se actualiza el scheduler basado en la métrica de validación, lo cual puede ajustar la tasa de aprendizaje u otros hiperparámetros según el rendimiento en el conjunto de validación.

4. **Seguimiento y Guardado de Mejores Modelos**: Si el rendimiento en el conjunto de validación mejora respecto al mejor registrado hasta el momento, se actualiza la mejor métrica y se guarda el estado del modelo (incluyendo los parámetros del modelo, el estado del optimizador, el scheduler y la mejor métrica) usando `save_checkpoint`. Si no hay mejora, se incrementa un contador que puede desencadenar el early stopping si se alcanza un límite de paciencia definido.

5. **Early Stopping**: Si no se observa mejora en la métrica de validación durante un número especificado de épocas (`args.patience`), el entrenamiento se detiene anticipadamente para evitar el sobreajuste y reducir el tiempo de cómputo.

6. **Registro de Métricas y Tiempos**: Se imprime la precisión de entrenamiento, la pérdida promedio de la época, la precisión de validación y el tiempo total de la época al final de cada época, proporcionando una visión en tiempo real del progreso del entrenamiento.

Finalmente, al concluir todas las épocas o al activarse el early stopping, se imprime la duración total del entrenamiento, ofreciendo una medida del tiempo que tomó entrenar el modelo.

In [26]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
    epoch_stats_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()
    
    for window_words, labels in train_loader:
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()
            
        #Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())
        
        #Get training metrics
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt, y_pred))
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #Get metric in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)
    
    #Get metric in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    metric_history.append(mean_epoch_metric)
    
    #update scheduler
    scheduler.step(tuning_metric)
    
    #Check for metric improvement
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve = 1
        
    save_checkpoint(
        {
            "epoch": epoch + 1,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_metric": best_metric,
        },
        is_improvement,
        args.savedir,
    )
    
    #Early stopping
    if n_no_improve >= args.patience:
        print("No improvement. Breaking out of loop.")
        
    print("Train acc: {}".format(mean_epoch_metric))
    print("Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}"
         .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time())))
    

print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.17303716155144017
Epoch [1/100], Loss: 5.5234 - Val accuracy: 0.1909 - Epoch time: 1711067898.82
Train acc: 0.18379570813079982
Epoch [2/100], Loss: 5.0786 - Val accuracy: 0.1449 - Epoch time: 1711067915.53
Train acc: 0.18953652884344996
Epoch [3/100], Loss: 4.8677 - Val accuracy: 0.1607 - Epoch time: 1711067934.89
Train acc: 0.1940113058892058
Epoch [4/100], Loss: 4.7005 - Val accuracy: 0.1809 - Epoch time: 1711067952.71
Train acc: 0.1977990800626682
Epoch [5/100], Loss: 4.5542 - Val accuracy: 0.2263 - Epoch time: 1711067968.51
Train acc: 0.1999077928534126
Epoch [6/100], Loss: 4.4211 - Val accuracy: 0.2167 - Epoch time: 1711067985.10
Train acc: 0.20232187512553731
Epoch [7/100], Loss: 4.3000 - Val accuracy: 0.2156 - Epoch time: 1711068002.23
Train acc: 0.2063271426706303
Epoch [8/100], Loss: 4.1793 - Val accuracy: 0.2279 - Epoch time: 1711068018.39
Train acc: 0.2080595574056964
Epoch [9/100], Loss: 4.0763 - Val accuracy: 0.2524 - Epoch time: 1711068034.29
Train acc: 0.21

## Evaluación de un modelo de lenguaje neuronal

Evaluar un modelo de lenguaje neuronal es un paso crucial después del entrenamiento para entender su rendimiento y cómo se generaliza a datos no vistos. La evaluación implica medir cuán bien el modelo predice o genera texto, utilizando métricas específicas y conjuntos de datos de prueba o validación. Aquí hay algunos aspectos clave para comenzar:

### 1. **Conjunto de Datos de Evaluación**:
Usualmente, se separa un conjunto de datos que no se utiliza durante el entrenamiento para evaluar el modelo. Este conjunto puede ser de validación (usado para ajustar hiperparámetros) o de prueba (usado para medir el rendimiento final del modelo). Es importante que el modelo no haya "visto" estos datos durante el entrenamiento para obtener una evaluación justa de su capacidad de generalización.

### 2. **Métricas de Evaluación**:
Dependiendo de la tarea específica del modelo de lenguaje, las métricas de evaluación pueden variar:
   - **Perplejidad**: Comúnmente usada en modelos de lenguaje, mide cuán "sorprendido" está el modelo por los datos de prueba, con valores más bajos indicando un mejor rendimiento.
   - **Precisión, Recall, y F1**: Usadas en tareas de clasificación, como análisis de sentimientos o clasificación de temas.
   - **BLEU, ROUGE, METEOR**: Usadas para evaluación en tareas de generación de texto, como traducción automática o resumen automático, donde se comparan las salidas del modelo con referencias humanas.

### 3. **Evaluación Cualitativa**:
Además de las métricas cuantitativas, es útil realizar una evaluación cualitativa del modelo, examinando las predicciones del modelo y cómo maneja varios casos de uso, errores comunes, y situaciones específicas del dominio. Esto puede incluir la revisión manual de las predicciones del modelo para evaluar su coherencia, relevancia y creatividad.

### 4. **Análisis de Errores**:
Identificar y analizar los errores cometidos por el modelo puede proporcionar insights valiosos sobre sus limitaciones y áreas para mejora. Esto puede implicar explorar casos donde el modelo se desempeña mal y tratar de entender las razones detrás de estos errores.

### 5. **Comparación con Modelos de Referencia**:
Es útil comparar el rendimiento de tu modelo con el de modelos de referencia o benchmarks en la misma tarea. Esto puede darte una idea de cómo se compara tu enfoque con el estado del arte o con enfoques más simples.

### 6. **Consideraciones Éticas y de Sesgo**:
En la evaluación de modelos de lenguaje, especialmente aquellos entrenados en grandes corpus de texto de internet, es importante considerar y evaluar posibles sesgos y problemas éticos en las predicciones del modelo. Esto puede incluir sesgos de género, raza, o cultural que el modelo podría haber aprendido de los datos de entrenamiento.

### 7. **Evaluación en Ambientes de Producción**:
Finalmente, si el modelo va a ser desplegado en un sistema en producción, es crucial realizar evaluaciones en un entorno que simule el uso real, ya que el comportamiento del modelo puede variar significativamente en condiciones del mundo real en comparación con un entorno de prueba controlado.

La evaluación de modelos de lenguaje es un proceso iterativo y multifacético que va más allá de simples métricas, abarcando evaluaciones cualitativas, comparativas, y éticas para obtener una comprensión completa del rendimiento y las aplicaciones del modelo.

---

Vamos a comparar la representación de las palabras, calculando las distancias y viendo que tan cerca están. 

In [27]:
def print_closest_words(embeddings, ngram_data, word, n):
    word_id = torch.LongTensor([ngram_data.w2id[word]])
    word_embed = embeddings(word_id)
    dists = torch.norm(embeddings.weight - word_embed, dim = 1).detach()
    lst = sorted(enumerate(dists.numpy()), key = lambda x: x[1])
    for idx, difference in lst[1:n+1]:
        print(ngram_data.id2w[idx], difference)

Cargamos el mejor modelo que se haya guardado y proseguimos para hacer una visualización

In [28]:
# Model with learned embeddings from scratch
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load("model/model_best.pt")["state_dict"])

print("-"*30)
print("Learned embeddings")
print("-"*30)
print_closest_words(best_model.emb, ngram_data, "jaja", 10)

------------------------------
Learned embeddings
------------------------------
<s> 10.224871
<unk> 10.376606
compras 10.435783
feliz 10.801502
👸 10.8067255
últimos 11.08122
#eliminatoriasconmebol 11.145872
chillón 11.239394
único 11.266181
teléfono 11.281281


In [29]:
def parse_text(text, tokenizer):
    # Tokeniza el texto y convierte cada palabra a minúsculas. Si la palabra está en el vocabulario (w2id), la usa;
    # de lo contrario, la reemplaza por el token "<unk>" para palabras desconocidas.
    all_tokens = [w.lower() if w.lower() in ngram_data.w2id else "<unk>" for w in tokenizer.tokenize(text)]
    
    # Convierte los tokens a sus índices numéricos correspondientes según el mapeo w2id en ngram_data.
    token_ids = [ngram_data.w2id[word.lower()] for word in all_tokens]
    
    return all_tokens, token_ids


def sample_next_word(logits, temperature=1.0):
    # Convierte los logits a un array de numpy y ajusta la "temperatura" de la predicción.
    logits = np.asarray(logits).astype("float64")
    preds = logits / temperature
    
    # Convierte los logits ajustados a probabilidades usando softmax.
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    # Muestrea un índice de palabra de la distribución de probabilidades.
    probas = np.random.multinomial(1, preds)
    return np.argmax(probas)


def predict_next_token(model, token_ids):
    # Convierte la lista de índices de tokens a un tensor de PyTorch y agrega una dimensión de lote.
    word_ids_tensor = torch.LongTensor(token_ids).unsqueeze(0)
    
    # Obtiene los logits de la predicción del modelo para la secuencia de tokens y los convierte a numpy.
    y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()
    
    # Muestra el índice de la siguiente palabra de la distribución de logits.
    y_pred = sample_next_word(y_raw_pred, 1.0)
    return y_pred


def generate_sentence(model, initial_text, tokenizer):
    # Obtiene tokens y sus índices del texto inicial.
    all_tokens, window_word_ids = parse_text(initial_text, tokenizer)
    
    # Genera hasta 100 palabras adicionales.
    for i in range(100):
        # Predice el índice de la siguiente palabra utilizando el modelo.
        y_pred = predict_next_token(model, window_word_ids)
        next_word = ngram_data.id2w[y_pred]  # Convierte el índice de palabra predicho a texto.
        all_tokens.append(next_word)  # Añade la palabra predicha a la lista de tokens.
        
        # Si se genera el token de fin de secuencia, detiene la generación.
        if next_word == "</s>":
            break
        else:
            # Actualiza la ventana de palabras para la siguiente predicción.
            window_word_ids.pop(0)  # Elimina la primera palabra.
            window_word_ids.append(y_pred)  # Añade la nueva palabra al final.
    
    # Une los tokens generados en una cadena de texto y la devuelve.
    return " ".join(all_tokens)


Se realiza la generación de texto utilizando el modelo de lenguaje neuronal previamente entrenado (`best_model`), comenzando con un texto inicial dado (`initial_tokens`). Veamos paso a paso el proceso:

1. **Definir el Texto Inicial**: La variable `initial_tokens` contiene el texto inicial desde el cual el modelo comenzará a generar texto. En este caso, parece que se utilizan tokens especiales de inicio `<s>` para indicar el comienzo de una nueva secuencia. La presencia de múltiples tokens `<s>` podría ser una convención para señalar el inicio de una secuencia, especialmente en modelos que utilizan n-gramas o contextos de longitud fija.

2. **Impresión de Encabezados**: Se imprimen líneas de guiones y el texto "Learned embeddings" como encabezado para indicar que a continuación se mostrarán los resultados de la generación de texto.

3. **Generación de Texto**: La función `generate_sentence` se utiliza para generar una secuencia de texto a partir del `initial_tokens` proporcionado. Esta función toma el modelo entrenado (`best_model`), el texto inicial, y un tokenizador (`tk`) para procesar el texto. La función realiza las siguientes tareas:
   - Tokeniza el texto inicial y obtiene los índices correspondientes de los tokens en el vocabulario del modelo (`parse_text`).
   - Utiliza el modelo para predecir la próxima palabra en la secuencia, basándose en los índices de tokens actuales (`predict_next_token`).
   - Añade la palabra predicha a la secuencia y actualiza el contexto de palabras para la próxima predicción.
   - Repite el proceso de predicción y actualización para generar hasta 100 palabras o hasta que se genere un token de fin de secuencia (`</s>`).
   - Devuelve el texto generado como una cadena de texto, uniendo los tokens generados.

4. **Impresión del Texto Generado**: Finalmente, el texto generado por la función `generate_sentence` se imprime en la consola. Este texto es una continuación del `initial_tokens` proporcionado, generado por el modelo `best_model` basándose en lo que ha aprendido durante el entrenamiento sobre la estructura y el contenido típicos del lenguaje en el dominio de los datos con los que fue entrenado.

En resumen, estas funciones y el código proporcionado demuestran cómo se puede utilizar un modelo de lenguaje neuronal entrenado para generar texto de manera autónoma, comenzando con un prompt o secuencia inicial dada, lo cual es útil en aplicaciones como la generación de texto creativo, la auto-completación de texto, y más.

In [31]:
initial_tokens = "<s><s><s>"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> se armó el joto mamón que <unk> en esta amargado y lo <unk> que son viejas a mi madre que consiente listo es tonta y soltar </s>


In [32]:
initial_tokens = "<s><s>estoy"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> estoy hasta la puta madre pero a los jotos madre el cuerpo cuando la misma está pero loca </s>


In [33]:
initial_tokens = "<s> saludos a"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> saludos a la novia <unk> tiene un genio pedo está desde el invierno pasado para ver el capitulo de <unk> haciendo súper " the queda <unk> <unk> a la verga chamaca </s>


In [34]:
initial_tokens = "<s> saludos a"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> saludos a <unk> como me hijo de mil putas <unk> tu forma de ser jamás <unk> siempre les <unk> <unk> así de <unk> amika no se les puede decir nada sin que <unk> </s>


In [35]:
initial_tokens = "yo opino que"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
yo opino que como no c puede mal <unk> 😤 </s>


La función `log_likelihood` calcula la log-verosimilitud de una secuencia de texto dada bajo un modelo de lenguaje entrenado. La log-verosimilitud es una medida de cuán probable es que el modelo haya generado una secuencia de texto dada.

In [36]:
def log_likelihood(model, text, ngram_model):
    # Transforma el texto dado en n-gramas (X) y las etiquetas objetivo (y) utilizando el modelo n-gram.
    X, y = ngram_data.transform([text])
    
    # Ignora los primeros dos n-gramas. Esto podría ser específico para cómo se estructura el texto o los n-gramas.
    X, y = X[2:], y[2:]
    
    # Convierte X en un tensor de PyTorch y agrega una dimensión de lote, preparándolo para el modelo.
    X = torch.LongTensor(X).unsqueeze(0)
    
    # Obtiene los logits del modelo para el texto transformado y luego los desconecta del grafo de cómputo.
    logits = model(X).detach()
    
    # Aplica softmax a los logits para obtener una distribución de probabilidades sobre el vocabulario.
    probs = F.softmax(logits, dim=1).numpy()
    
    # Calcula la log-verosimilitud sumando los logaritmos de las probabilidades de las palabras reales (y)
    # según lo predicho por el modelo.
    return np.sum([np.log(probs[i][w]) for i, w in enumerate(y)])


Probamos el modelo. La log-verosimilitud es una medida de cuán probable es que una secuencia de texto dada sea generada por un modelo de lenguaje. Valores más altos de log-verosimilitud indican que el modelo asigna una mayor probabilidad a la secuencia de texto observada, lo que sugiere que el modelo piensa que es más "verosímil" o probable.

Cuando calculas la log-verosimilitud, sumas los logaritmos de las probabilidades asignadas a las palabras reales. Dado que estas probabilidades están entre 0 y 1, sus logaritmos son negativos. Por lo tanto, una log-verosimilitud más alta (menos negativa) significa que las probabilidades asociadas a las palabras observadas son más altas, y el modelo considera que la secuencia es más probable. Una log-verosimilitud muy negativa indica que el modelo asigna muy bajas probabilidades a las palabras observadas, lo que sugiere que el modelo considera que esa secuencia de texto es poco probable.

Así que, en resumen, buscas maximizar la log-verosimilitud. Valores más cercanos a cero o menos negativos son mejores, ya que indican que el modelo asigna mayores probabilidades a las secuencias de texto observadas.

In [37]:
print("log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", ngram_data))

log likelihood:  -24.262943


In [38]:

print("log likelihood: ", log_likelihood(best_model, "la natural estmos clase en de de lenguaje procesamiento", ngram_data))

log likelihood:  -40.692436


## Estructuras sintácticas correctas

El siguiente código explora las diferentes permutaciones de una secuencia de palabras dada y las evalúa usando el modelo de lenguaje (`best_model`) para determinar la log-verosimilitud de cada permutación. A continuación, detallo lo que hace cada parte del código y luego te doy una introducción a las estructuras sintácticas correctas.

### Exploración de Permutaciones

1. **Creación de la Lista de Palabras**: La frase "sino gano me voy a la chingada" se divide en palabras individuales para formar `word_list`.

2. **Generación de Permutaciones**: Utilizando `permutations` de `itertools`, el código genera todas las posibles permutaciones de la lista de palabras. Cada permutación se une en una cadena de texto separada por espacios y se almacena en `perms`.

3. **Evaluación de las Permutaciones**: Para cada permutación, se calcula la log-verosimilitud usando la función `log_likelihood`, pasando el `best_model`, la permutación de texto, y `ngram_data`. Se crean tuplas de log-verosimilitud y texto de permutación.

4. **Ordenación y Presentación de Resultados**: Las tuplas se ordenan por log-verosimilitud de manera descendente. Luego, el código imprime las 5 permutaciones con mayor log-verosimilitud y las 5 permutaciones con menor log-verosimilitud para mostrar cuáles secuencias de palabras son consideradas más y menos probables por el modelo.

### Estructuras Sintácticas Correctas

Las estructuras sintácticas en un idioma se refieren a cómo se organizan las palabras y frases para formar oraciones significativas y gramaticalmente correctas. La sintaxis incluye reglas sobre el orden de las palabras, la concordancia entre sujetos y verbos, el uso correcto de los tiempos verbales, y la estructuración de frases complejas, entre otros aspectos.

En lenguajes como el español o el inglés, la estructura sintáctica típica sigue un patrón de "Sujeto-Verbo-Objeto" (SVO), aunque puede variar dependiendo del propósito y estilo de la oración. Por ejemplo, en la frase "Yo como manzanas", "Yo" es el sujeto, "como" el verbo, y "manzanas" el objeto.

La sintaxis también involucra el uso de conectores, preposiciones, artículos y otros elementos que ayudan a unir las palabras en frases coherentes y estructuradas. Una comprensión profunda de la sintaxis es crucial para la construcción de oraciones que no solo sean gramaticalmente correctas, sino que también transmitan claramente la intención del hablante o escritor.

En el contexto del fragmento de código proporcionado, el modelo de lenguaje intenta capturar algunas de estas reglas sintácticas implícitamente a través del aprendizaje de las probabilidades de secuencias de palabras basado en el corpus de entrenamiento. Sin embargo, al generar todas las permutaciones posibles de una lista de palabras, muchas de las secuencias resultantes serán sintácticamente incorrectas o carecerán de sentido, ya que no todas las combinaciones de palabras forman oraciones coherentes o gramaticalmente correctas en un idioma natural.

In [39]:
from itertools import permutations
from random import shuffle

word_list = "sino gano me voy a la chingada".split(" ")
perms = [" ".join(perm) for perm in permutations(word_list)]
#print(len(perms))
print("-"*50)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[:5]:
    print(p, t)
print("-"*50)  
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[-5:]:
    print(p, t)

--------------------------------------------------
-26.891914 sino gano a la chingada me voy
-27.684141 gano sino me voy a la chingada
-28.632072 gano voy a me la chingada sino
-29.288256 gano sino me a voy la chingada
-29.987896 gano voy me a la chingada sino
--------------------------------------------------
-97.33807 la a voy chingada gano me sino
-97.442696 a me sino gano voy chingada la
-98.99779 me a chingada sino voy gano la
-99.170105 la a sino gano voy chingada me
-100.42936 la a chingada gano me sino voy
